## Pytorch Tutorials : Quick Start
- [Pytorch Tutorials](https://pytorch.org/tutorials/)
- [Quick Start](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

In [ ]:
%matplotlib inline

### Quick Start
- DataSet (`torch.utils.data.Dataset`) : __len__ 함수와 __getitem__ 함수를 지닌 어떤 것이라도 될 수 있으며, 이 함수들을 인덱싱하기 위한 방법으로서 사용됨
- DataLoader (`torch.utils.data.DataLoader`) : 배치 관리를 담당, 모든 Dataset으로부터 DataLoader를 생성할 수 있음  

기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

TorchVision Dataset에는 각각 샘플과 레이블을 수정하기 위한 변환과 target_transform의 두 가지 인수가 포함되어 있음

In [ ]:
# Pytorch의 open dataset인 fashionMNIST로부터 train set다운
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Pytorch의 open dataset인 fashionMNIST로부터 test set다운
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


DataLoader에 Dataset을 전달한다. 이는 데이터 세트에 적용할 수 있는 기능을 제공하며 자동 배치, 샘플링, 셔플링 및 다중 처리 데이터 로드를 지원합니다. 여기서 배치 크기를 64로 정의한다. 즉, 데이터 로더의 각 요소는 64개의 특징과 레이블의 배치를 반환한다.

In [ ]:
batch_size = 64

# 데이터 로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


### Creating Models

PyTorch에서 신경망을 정의하기 위해 nn에서 상속되는 클래스를 만든다. 우리는 `__init__` 함수에서 네트워크의 계층을 정의하고 전진 함수에서 데이터가 네트워크를 통과하는 방법을 지정한다.

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the Model Parameters

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

단일 훈련 루프에서 모델은 훈련 데이터 세트에 대한 예측을 수행하고 예측 오류를 역전파하여 모델의 매개 변수를 조정한다.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303149  [    0/60000]
loss: 2.294281  [ 6400/60000]
loss: 2.263092  [12800/60000]
loss: 2.259313  [19200/60000]
loss: 2.254132  [25600/60000]
loss: 2.210443  [32000/60000]
loss: 2.232525  [38400/60000]
loss: 2.190079  [44800/60000]
loss: 2.184380  [51200/60000]
loss: 2.149513  [57600/60000]
Test Error: 
 Accuracy: 28.5%, Avg loss: 2.140794 

Epoch 2
-------------------------------
loss: 2.154680  [    0/60000]
loss: 2.148116  [ 6400/60000]
loss: 2.081002  [12800/60000]
loss: 2.104912  [19200/60000]
loss: 2.058291  [25600/60000]
loss: 1.986273  [32000/60000]
loss: 2.032925  [38400/60000]
loss: 1.941413  [44800/60000]
loss: 1.951164  [51200/60000]
loss: 1.881139  [57600/60000]
Test Error: 
 Accuracy: 55.5%, Avg loss: 1.875404 

Epoch 3
-------------------------------
loss: 1.914312  [    0/60000]
loss: 1.883507  [ 6400/60000]
loss: 1.762080  [12800/60000]
loss: 1.813229  [19200/60000]
loss: 1.697275  [25600/60000]
loss: 1.651208  [32000/600

### Saving Models

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading Models

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
